# Create Dataset

The original cell dataset is viper_ds.csv located in the ./dataset directory and requires 7GB of space. 
To load the entire viper_ds dataset more than 16GB of memory are required. Since my local machine doesn't meet the requirements, and for the purpose of this test, we will work with subsamples. 

These subsamples contain n [10000,100000,1000000] samples.

The original vipe_ds.csv contains roughtly 36 Mio rows of cell data, where each row represents a single cell.
The first half (18Mio) are labeled as bad cells, since these are the cells which interfere the simulation and cause problems during the sumulation convergence.

The second half (18Mio) are labeled as good cells, which are all the cells which cause no problems during the sumulation.


In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import csv
import time

First we specify a list of samplesizes that will create our subsamples.

We use pandas to read the original dataset. Pandas offers a parameter to limit the number of rows to be read from the file. (nrows)

We retrieve a subsample from the section of bad cells and a subsample from the section of good cells. To acheive this, we skip the header from the section of bad cells and then start reading until we satisfy the desired samplesize.

We repeat this step and skip 20Mio rows from the entire set to be somewhere located within the section of good cells and start counting from here. 

With this approach we can make sure, that we collect n/2 bad samples and n/2 good samples from the main dataset.


After mergin the two sets of bad and good cells we save the DataFrame to the local disk as a csv file. This will be done via the df.to_csv() command.

. 

Apache Arrow is an ideal in-memory transport layer for data that is being read or written with Parquet files. 
The Apache Parquet project provides a standardized open-source columnar storage format for use in data analysis systems.

https://arrow.apache.org/docs/python/parquet.html



Next we also want to save the combined DataFrame to the Apache Parquet Format.
This requires the DataFrame to be converted to the pyarrow.Table object. We can achieve this via the pa.Table.from_pandas() function.


The PyArrow Table type is not part of the Apache Arrow specification, but is rather a tool to help with wrangling multiple record batches and array pieces as a single logical dataset.


Finally, we write the table to disk.

In [3]:
n = [10000,100000,1000000]

header = pd.read_csv("./dataset/viper_ds.csv", nrows=1)

for nc in n:
    df_badcells = pd.read_csv("./dataset/viper_ds.csv", skiprows= 1,nrows=nc/2)
    df_goodcells = pd.read_csv("./dataset/viper_ds.csv", skiprows= 20000000,nrows=nc/2)
    
    df_badcells.columns = header.columns
    df_goodcells.columns = header.columns
    
    combined = pd.concat([header, df_badcells,df_goodcells])
    combined.reset_index(drop=True)
        
    ## write dataframe to csv
    combined.to_csv("./dataset/cell_data_"+str(nc)+".csv", index=False)
    
    ## wirte dataframe to parquet
    py_arrow_combined = pa.Table.from_pandas(combined)
    pq.write_table(py_arrow_combined, "./dataset/cell_data_"+str(nc)+"_arrow.parquet")
    

Now we test the performance of reading the new parquet files in comparison to the csv files.

For this test we apply four functions:

 - pandas read_csv()
 - pyarrow.parquet read_table()
 - pyarrow.parquet read_pandas()
 - pyarrow csv()
 
We will measure the time duration reading in the files.

For reading a parquet file, pyarrow offers two function. (read_table(), read_pandas())

In practice, a Parquet dataset may consist of many files in many directories. We can read a single file back with read_table.

When reading a file that used a Pandas dataframe as the source, we use read_pandas

In [4]:
def load_dfs(nc):
    l_times = []
    
    # standard pandas read csv
    start = time.time()
    header = pd.read_csv('./dataset/cell_data_'+str(nc)+'.csv')
    end = time.time()
    l_times.append(end-start)
    
    # apyrrow read parquet table file
    start = time.time()
    table2 = pq.read_table('./dataset/cell_data_'+str(nc)+'_arrow.parquet').to_pandas()
    end = time.time()
    l_times.append(end-start)
    
    # pyarrow read parquet file from a pandas original source
    start = time.time()
    table3 = pq.read_pandas('./dataset/cell_data_'+str(nc)+'_arrow.parquet').to_pandas()
    end = time.time()
    l_times.append(end-start)
    
    #pyarrow read csv
    start = time.time()
    table4 = csv.read_csv('./dataset/cell_data_'+str(nc)+'.csv').to_pandas()
    end = time.time()
    l_times.append(end-start)
        
    return pd.Series(l_times)


In [5]:
timings_df = pd.DataFrame()
for nc in n:
    timings_df = pd.concat([timings_df, load_dfs(nc)], axis = 1)

index = ['pandas read_csv', 'pyarrow from table', 'pyarrow from pandas', 'pyarrow read_csv']

timings_df.index = index
timings_df.columns = n

timings_df

,10000,100000,1000000
pandas read_csv,0.026139,0.225137,2.013270
pyarrow from table,0.072070,0.054328,0.120114
pyarrow from pandas,0.007632,0.015180,0.089052
pyarrow read_csv,0.014740,0.051877,0.516226
